In [52]:
# 시간 다 재맘대로임. 이것 처리 위해서는( 도서관 2개 -> 일괄 오후 2시 , 도서관 3개 -> 
# 최대한 상영일시에서 뽑아보고 없으면 상영시간에서 뽑는다.<한곳은 걍 독립영화텝만 뽑겠슴 >   )  
# ( 기존의 런타임 = 상영시간은 이번엔 안뽑음. )  

import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib import parse


libGroup = "eplib"
libGroup = "enlib"
libGroup = "jsplib"
libGroup = "ealib"
libGroup = "gsvlib"


url = "http://www."+ libGroup + ".or.kr/culture/movie.asp"
if libGroup == "gsvlib":
    url = url + "?mode=list&gubun=3"
request = requests.get(url)
response = request.text
soup = BeautifulSoup(response, 'lxml')
movie_tables = soup.select('div.tableF > ul.clear > li > dl.book_exp') # 영화 없으면 아예 아래 포문 안들어가게 하겠슴.


for movie_table in movie_tables:
    title = ""  # 제목
    when = ""  # 일시
    when_date_year = 0
    when_date_month = 0
    when_date_day = 0
    when_time_hour = 0
    when_time_minuite = 0
    runtime = 0  # 런타임
    place = ""  # 장소

    # 제목 따로 dt테그라 먼저 찾겠슴 
    title_pre = movie_table.select_one('dt')
    if title_pre:
        title = title_pre.get_text(strip=True)
        if re.findall('\((.*)\)',title):       # 이월 (독립영화),(더빙),(자막) 등 빼주기위함. 
            title_before_pre = re.findall('(.*)\(.*\)',title)
            if title_before_pre:
                title = title_before_pre[0]
        print(f'title: {title}')
    else:
        print('영화제목없음')
        continue# 타이틀 없으면 다음 영화로 넘어감. 
        
    # 나머지정보들 묶여서 한번에 크롤링
    item_pres = movie_table.select('dd > p')
    for item_pre in item_pres:
        item_name_pre = item_pre.get_text()

        if "상영일시" in item_name_pre:

            when = item_name_pre
            print(f'when: {when}')

            when_date_year_pre = re.findall('(\d\d\d\d)', when)
            if when_date_year_pre:
                when_date_year = when_date_year_pre[0]
            when_date_month_pre = re.findall('\d\d\d\d\s*-\s*(\d{1,2})', when)
            if when_date_month_pre:
                when_date_month = when_date_month_pre[0]
            when_date_day_pre = re.findall('\d\d\d\d\s*-\s*\d{1,2}\s*-\s*(\d{1,2})', when)
            if when_date_day_pre:
                when_date_day = when_date_day_pre[0]
            
            
            
            if libGroup in ["eplib", "enlib"]: # 이 두 사이트는 공지사항에만 몇시인지 따로 명시해놓음
                when_time_hour = "14"
                when_time_minuite = "0" 
                
            # 18:00 이런식으로 표현시 
            elif re.findall('(\d{1,2})\s*:', when):
                when_time_hour_pre= re.findall('(\d{1,2})\s*:', when)
                if when_time_hour_pre:
                    when_time_hour = when_time_hour_pre[0]
                when_time_minuite_pre = re.findall('\d{1,2}\s*:\s*(\d{1,2})\s*', when)
                if when_time_minuite_pre:
                    when_time_minuite = when_time_minuite_pre[0]
            # 18시 30분 이런식으로 표현시 
            elif re.findall('\d+\s*시\s*(\d+)\s*분', when):
                when_time_hour_pre = re.findall('(\d+)\s*시\s*\d+\s*분', when)
                if when_time_hour_pre:
                    when_time_hour = when_time_hour_pre[0]
                when_time_minuite_pre = re.findall('\d+\s*시\s*(\d+)\s*분', when)
                if when_time_minuite_pre:
                    when_time_minuite = when_time_minuite_pre[0]

            # 18시 이런식으로 표현될땐 ~
            elif re.findall('(\d+)\s*시', when):
                when_time_hour_pre = re.findall('(\d+)\s*시', when)
                if when_time_hour_pre:
                    when_time_hour = when_time_hour_pre[0]

            print(f'when_date_year: {when_date_year}')
            print(f'when_date_month: {when_date_month}')
            print(f'when_date_day: {when_date_day}')
            print(f'when_time_hour: {when_time_hour}')
            print(f'when_time_minuite: {when_time_minuite}')
            
            
        elif "장소" in item_name_pre:
            place_pre = item_name_pre.split('장소')
            if place_pre:
                place = place_pre[1]
                print(f'place: {place}')
        
        # 상영시간에 상영 일시 적힌곳있어서... 상영일시에서 시간,분 0,0인것에한해서  
        # 여기서 상영일시 시간, 분을 얻어보겠다. (런타임은 일괄적으로 안뽑겠다. 
        # 여기서도 안뽑히면 0시 0분으로 갈수밖에 없음. 
        elif "상영시간" in item_name_pre:
            when_time_hour_pre =""
            when_time_minute_pre=""
        
            # 18:00 이런식으로 표현시 
            if re.findall('(\d{1,2})\s*:', item_name_pre):
                when_time_hour_pre= re.findall('(\d{1,2})\s*:', item_name_pre)
                when_time_minuite_pre = re.findall('\d{1,2}\s*:\s*(\d{1,2})\s*', item_name_pre)
            # 18시 30분 이런식으로 표현시 
            elif re.findall('\d+\s*시\s*(\d+)\s*분', item_name_pre):
                when_time_hour_pre = re.findall('(\d+)\s*시\s*\d+\s*분', item_name_pre)
                when_time_minuite_pre = re.findall('\d+\s*시\s*(\d+)\s*분', item_name_pre)

            # 18시 이런식으로 표현될땐 ~
            elif re.findall('(\d+)\s*시', item_name_pre):
                when_time_hour_pre = re.findall('(\d+)\s*시', item_name_pre)
            
            
            # 위에 규칙대로 상영시간에서 hour뽑혔다면  -> 상영시간 (시간,분) 여기 적혀있는것. -> 이것으로최신화 
            if when_time_hour_pre:
                when_time_hour = when_time_hour_pre[0]
                print(f'when_time_hour: {when_time_hour}  (상영시간에서 추출해서 이것으로변경)')
            if when_time_minute_pre:
                when_time_minuite = when_time_minuite_pre[0]
                print(f'when_time_minuite: {when_time_minuite} (상영시간에서 추출해서 이것으로변경)') # 0에서 ->0 바귄것이면 안뜸. 
                    
            
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')









title: 이월
when: 상영일시2019-06-11
when_date_year: 2019
when_date_month: 06
when_date_day: 11
when_time_hour: 0
when_time_minuite: 0
when_time_hour: 7  (상영시간에서 추출해서 이것으로변경)
place: 3층 청소년힐링캠프
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2
title: 어른이 되면
when: 상영일시2019-06-25
when_date_year: 2019
when_date_month: 06
when_date_day: 25
when_time_hour: 0
when_time_minuite: 0
when_time_hour: 7  (상영시간에서 추출해서 이것으로변경)
place: 3층 청소년힐링캠프
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2
